<div align="center">
  <img src="assets/Day5.png" alt="Databricks 14 Days AI Challenge - Day 05" width="800"/>
</div>

## DAY 5 (13/01/26) – Delta Lake Advanced

### 📚 Learning Objectives
Yesterday we built the foundation. Today, we master the features that make Delta Lake "production-grade":
* **Time Travel:** Querying older versions of data for auditing or rollback (like Git for data). 
* **Incremental MERGE:** Efficiently updating changing data (SCD Type 1/2) without rewriting the full table.
* **Optimization:** Using `OPTIMIZE` and `ZORDER` to speed up queries by organizing file layout.
* **Maintenance:** Using `VACUUM` to clean up stale files and manage storage costs.

### 🚀 Strategy: The "Audit & Optimize" Pipeline
1.  **Simulate Updates:** Create a batch of "corrected" data (e.g., updating incorrect prices) and new records.
2.  **Upsert (Merge):** Apply these changes to our Silver table using the `MERGE` command.
3.  **Audit (Time Travel):** Compare the table "Before" vs. "After" to verify the changes.
4.  **Tune:** Reorganize the physical file layout to make future queries faster.

###Setup & Load
**Task**: Define paths and initialize the Delta Table object.

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, lit, current_timestamp

# Define the path to our existing Silver table (created in Day 4)
delta_path = "/Volumes/workspace/ecommerce/ecommerce_data/delta/events_bronze"

# Initialize DeltaTable object (Entry point for advanced DML operations)
delta_table = DeltaTable.forPath(spark, delta_path)

print(f"✅ Delta Table loaded from: {delta_path}")
print(f"   Current Version: {delta_table.history().select('version').first()[0]}")

In [0]:
# Inspect the schema and sample data of the target Delta table
df_target = spark.read.format("delta").load(delta_path)
df_target.printSchema()
df_target.show(5)

###Incremental MERGE (The Upsert)
**Task**: Implement incremental MERGE. 

**Concept**: We simulate a scenario where we found "Data Errors" in yesterday's batch (e.g., negative prices) and need to fix them while simultaneously adding new real-time events.

In [0]:
# 1. CREATE "UPDATES" DATASET
# Scenario: 
# - User 563509121 had a transaction with price 100. We need to correct it to 150.
# - We also have 1 completely new incoming event.

updates_data = [
    # Existing Row (Update Case): Matching keys (user_id, event_time, product_id)
    ("2019-11-01 00:00:10", "purchase", 1004258, 563509121, "electronics.smartphone", "apple", 150.00, "Nov"),
    # New Row (Insert Case)
    ("2020-01-01 10:00:00", "purchase", 9999999, 111111111, "test.category", "test_brand", 50.00, "Jan")
]

# Create DataFrame with same schema as target
columns = ["event_time", "event_type", "product_id", "user_id", "category_code", "brand", "price", "month"]
df_updates = spark.createDataFrame(updates_data, columns)

# Add missing columns to match target schema
from pyspark.sql.functions import lit
if "category_id" not in df_updates.columns:
    df_updates = df_updates.withColumn("category_id", lit(None).cast("long"))
if "user_session" not in df_updates.columns:
    df_updates = df_updates.withColumn("user_session", lit(None).cast("string"))

# Reorder columns to match target table
target_columns = ["event_time", "event_type", "product_id", "category_id", "category_code", "brand", "price", "user_id", "user_session", "month"]
df_updates = df_updates.select(target_columns)

print("--- Incoming Updates Batch ---")
df_updates.show()

# 2. PERFORM MERGE
# Logic: Match on Composite Key (User + Time + Product)
delta_table.alias("target").merge(
    df_updates.alias("source"),
    """
    target.user_id = source.user_id AND 
    target.event_time = source.event_time AND 
    target.product_id = source.product_id
    """
).whenMatchedUpdateAll( # If key exists, update all columns (Corrects the price)
).whenNotMatchedInsertAll( # If key doesn't exist, insert new row
).execute()

print("✅ MERGE Operation Complete (Upsert executed).")

###Time Travel (Querying History)
* **Task**: Query historical versions. 
* **Concept**: We will verify our update by querying the table at version 0 (original) and version 1 (after merge).

In [0]:
# ---------------------------------------------------------
# STEP 3: TIME TRAVEL AUDIT
# Goal: Compare the "Before" (v0) and "After" (vLatest) states.
# Note: We add safety checks because the user might not have existed in v0!
# ---------------------------------------------------------

# 1. READ LATEST VERSION (Current State)
df_latest = spark.read.format("delta").load(delta_path)

# Safe extraction helper
def get_price_safe(df, uid, version_label):
    row = df.filter(col("user_id") == uid).select("price").first()
    if row:
        return row[0]
    else:
        return "NOT FOUND (Did not exist)"

# Target User ID (The one we updated/inserted in the previous step)
target_user_id = 563509121

# Get Prices
price_latest = get_price_safe(df_latest, target_user_id, "Latest")

# 2. READ OLD VERSION (Time Travel to Version 0)
# We use 'versionAsOf' to travel back to the very first write
df_old = spark.read.format("delta").option("versionAsOf", 0).load(delta_path)
price_old = get_price_safe(df_old, target_user_id, "v0")

print(f"🕵️ TIME TRAVEL AUDIT for User {target_user_id}:")
print(f"   Original Price (v0):      {price_old}")
print(f"   Updated Price  (vLatest): {price_latest}")

# 3. VISUALIZE HISTORY
# This shows the transaction log (who changed what and when)
display(delta_table.history().select("version", "timestamp", "operation", "operationMetrics"))

###Performance Optimization (Z-ORDER)
**Task**: Optimize tables. 

**Concept**: Z-Ordering is a technique to co-locate related information in the same set of files. Since we often filter by event_type (e.g., "purchase") and brand, clustering data by these columns allows Spark to skip huge chunks of irrelevant files during queries.

In [0]:
# Run OPTIMIZE and ZORDER BY high-cardinality query columns
# Note: This triggers a Spark job that compacts small files and reorganizes data
spark.sql(f"""
    OPTIMIZE delta.`{delta_path}`
    ZORDER BY (event_type, brand)
""")

print("✅ Optimization Complete. Small files compacted and Z-Ordered.")

### Storage Cleanup (VACUUM)
**Task**: Clean old files. 

**Concept**: When we "overwrite" or "update" data, Delta doesn't delete the old file immediately (to allow Time Travel). VACUUM permanently deletes files no longer referenced by a Delta log older than the retention period.

In [0]:
# ⚠️ SAFETY CHECK: 
# By default, Delta prevents vacuuming files newer than 7 days (168 hours) to prevent 
# corrupting long-running queries. We will respect this safe default.

# Run VACUUM with default retention
# dryRun=True allows us to see what WOULD be deleted without actually deleting it
deleted_files = delta_table.vacuum(retentionHours=168)

print("✅ VACUUM completed (Retention: 7 Days).")
# Note: If you really need to delete immediate files (DEV only), you would use:
# spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
# delta_table.vacuum(0)

### 🧠 Key Learnings & Takeaways
* **Merge > Rewrite:** We used `MERGE` to handle "late-arriving updates" efficiently. This is critical for Change Data Capture (CDC) pipelines.
* **Time Travel is Native:** We didn't need to build a separate history table. Delta Lake maintained the "Before" state automatically, allowing us to audit changes easily.
* **Z-Ordering:** We learned that `OPTIMIZE` isn't just about file size—it's about *sorting* data (Z-Ordering) to maximize "Data Skipping" performance during reads.
* **Storage Hygiene:** `VACUUM` is essential for controlling cloud costs, but it must be used carefully (it removes the ability to Time Travel back past the retention period).